<a href="https://colab.research.google.com/github/Marzieh777/Fraud-Detection-Crime-Rating/blob/master/fraud_detection_OneClassSvm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import packages
import pandas as pd
import numpy as np
import gzip
from sklearn.base import TransformerMixin


from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path_data= '/content/drive/My Drive/Colab Notebooks/fraud_prep.csv.gz'
fraud_df = pd.read_csv(path_data, compression='gzip', header=0, sep=',')

In [3]:
#NaN values for categorical and numerical:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

fraud_df_clean = DataFrameImputer().fit_transform(fraud_df)
print(len(fraud_df_clean))


284807


In [4]:
fraud_df_clean.head()

,Unnamed: 0,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,V29,V30,V31
0,0,0.0,0.012453,-0.072781,-1.127901,0.001365,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0,"4820 Sunrise Highway\nSayville, NY 11782\n(40....","JPMorgan Chase Bank, National Association",Suffolk
1,1,0.0,1.191857,0.266151,-1.127901,0.001365,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0,"51 20 Northern Boulevard\nWoodside, NY 11377\n","JPMorgan Chase Bank, National Association",Nassau
2,2,1.0,-1.358354,-1.340163,-1.127901,0.001365,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0,"103 Main Street\nCooperstown, NY 13326\n(42.70...",TD Bank N.A.,Erie
3,3,1.0,-0.966272,-0.185226,-1.127901,0.001365,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0,"2255 East Ridge Road\nRochester, NY 14622\n(43...",Santander Bank,Ulster
4,4,2.0,-1.158233,0.877737,-1.127901,0.001365,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0,"2872 Broadway\nNew York, NY 10025\n(40.805224,...",Chemung Canal Trust Company,Oswego


#Exploring data

In [5]:
CATEGORICAL_INPUT_FEATURES = ['V{}'.format(i) for i in range(30, 32)]
ALL_FEATURES = list(fraud_df_clean)

INPUT_FEATURES = ALL_FEATURES.copy()
INPUT_FEATURES.remove('Class') #target
INPUT_FEATURES.remove('V29') #seems not important
print(INPUT_FEATURES)
NUMERICAL_INPUT_FEATURES = [feature for feature in INPUT_FEATURES if feature not in CATEGORICAL_INPUT_FEATURES]
print(NUMERICAL_INPUT_FEATURES)
print(CATEGORICAL_INPUT_FEATURES)
fraud_df_clean.Class.value_counts()

['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'V30', 'V31']
['Unnamed: 0', 'Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']
['V30', 'V31']


0    284315
1       492
Name: Class, dtype: int64

# Test train split

In [6]:
#splitting data into train and test:
from sklearn.model_selection import train_test_split
import numpy as np

df_train, df_test = train_test_split(fraud_df_clean, test_size=0.3, random_state=1333)
X_train= df_train.drop('Class',axis =1)
X_test = df_test.drop('Class',axis =1)

y_train =df_train.Class
y_test =df_test.Class
print(len(y_train))
print(len(X_test))


199364
85443


#Encode categorical features v30, v31


In [7]:
# encode categorical attributes into a binary one-hot encoded representation in test data and train data
def encode_cat_features(cat_cols, train_data,test_data):
  train_encoded = pd.get_dummies(train_data[cat_cols], prefix_sep="__", columns=cat_cols)
  encoded_columns = list(train_encoded.columns[:])
  #print(encoded_columns)
  cat_dummies = [col for col in train_encoded if "__" in col and col.split("__")[0] in cat_cols]
  test_encoded = pd.get_dummies(test_data[cat_cols], prefix_sep="__", columns=cat_cols)
  # Remove additional columns
  for col in test_encoded.columns:
    if ("__" in col) and (col.split("__")[0] in cat_cols) and col not in cat_dummies:
      print("Removing additional feature {}".format(col))
      test_encoded.drop(col, axis=1, inplace=True)
  #add missing cols
  for col in cat_dummies:
    if col not in test_encoded.columns:
      print("Adding missing feature {}".format(col))
      test_processed[col] = 0
  test_encoded = test_encoded[encoded_columns]
  return (train_encoded,test_encoded)

X_train_cat, X_test_cat = encode_cat_features(CATEGORICAL_INPUT_FEATURES,X_train, X_test)
X_train_num = X_train[NUMERICAL_INPUT_FEATURES]
X_test_num = X_test[NUMERICAL_INPUT_FEATURES]

df_train= pd.concat([X_train_cat,X_train_num, y_train], axis = 1)
df_test = pd.concat([X_test_cat, X_test_num, y_test], axis = 1)

#Get a portion of data due to high number of data 
df_train_rd = df_train.sample(frac =.3)
df_test_rd= df_test.sample(frac = .3)

print(len(df_train_rd))
print(len(df_test_rd))

59809
25633


#Performing OneClassSVM :


In [8]:
from sklearn.svm import OneClassSVM 
import numpy as np
#print(len(train_data.Class))
df_train_norm = df_train_rd[df_train_rd.Class==0]
X_train_norm = df_train_norm.drop('Class', axis = 1)

#outliers
df_train_outliers = df_train_rd[df_train_rd.Class==1] 

outlier_prop = len(df_train_outliers) / len(df_train_norm) 
#svm = OneClassSVM(kernel='rbf', nu=outlier_prop, gamma=0.000001) 

#train on normal data
svm = OneClassSVM(kernel='rbf', gamma=0.000001)
svm.fit(X_train_norm)

y_true = pd.DataFrame(df_test_rd.Class.values)
test = df_test_rd.drop('Class', axis = 1)

#predict on test data
fraud_pred = svm.predict(test)

# Check the number of outliers predicted by the algorithm
unique, counts = np.unique(fraud_pred, return_counts=True)
print (np.asarray((unique, counts)).T)

#Convert Y-test and fraud_pred to dataframe for ease of operation
fraud_pred = pd.DataFrame(fraud_pred)
fraud_pred= fraud_pred.rename(columns={0: 'prediction'})
print(y_test.shape)
print(fraud_pred.shape)

[[   -1 12917]
 [    1 12716]]
(85443,)
(25633, 1)


#Display Results

In [14]:
##Performance check of the model
TP = FN = FP = TN = 0
for j in range(len(y_true)):
    if y_true[0][j]== 0 and fraud_pred['prediction'][j] == 1:
        TP = TP+1
    elif y_true[0][j]== 0 and fraud_pred['prediction'][j] == -1:
        FN = FN+1
    elif y_true[0][j]== 1 and fraud_pred['prediction'][j] == 1:
        FP = FP+1
    else:
        TN = TN +1
print (TP,  FN,  FP,  TN)
print('#=================')
# Performance Matrix

precision = TP/(TP+FP)
print ("Precision OCSVM: ",precision)
print('#==================')
true_Neg = TN/(TN+FP)
print ("specificity OCSVM: ", true_Neg)
print('#====================')
F1_Score = 2*TP/(2*TP+FP+FN)
print("F1 Score OCSVM", F1_Score)
print("#===================")

accuracy = (TP+TN)/(TP+FN+FP+TN)
print ("Accuracy OCSVM", accuracy)
print('#=====================')

recall = TP/(TP+FN)
print ("Recall OCSVM: ", recall)

12702 12877 14 40
#=================
Precision OCSVM:  0.9988990248505819
#==================
specificity OCSVM:  0.7407407407407407
#====================
F1 Score OCSVM 0.663376419898159
#===================
Accuracy OCSVM 0.49709359029376193
#=====================
Recall OCSVM:  0.49657922514562725
